In [24]:
import pandas as pd 
import json
import numpy as np

In [25]:
teams = ["Broncos", "Roosters", "Wests Tigers", "Rabbitohs", "Storm", "Eels", "Raiders", "Knights", "Dragons", "Sea Eagles", "Panthers", "Sharks", "Bulldogs", "Dolphins", "Titans", "Cowboys", "Warriors"]
variables =["Win", "Defense", "Attack", "Margin", "Home", "Versus"]

In [26]:
with open('nrl_data.json', 'r') as file:
    data = json.load(file)
    data = data['NRL']
    
df = pd.DataFrame(columns=[f"{team} {variable}" for team in teams for variable in variables])



In [27]:
all_store = []
for round in range(0, 21):
    round_data = data[round][str(round+1)]
    
    # Create an empty feature array 
    round_store = np.zeros([len(teams)*len(variables)], dtype=int)
    round_teams = []
    for game in round_data:
        h_team = game['Home']
        h_score = int(game['Home_Score'])
        a_team = game['Away']
        a_score = int(game['Away_Score'])
        
        # win or lose
        h_team_win = h_score >= a_score
        a_team_win = a_score >= h_score
        
        # home
        h_home = 1
        a_home = 0
        
        # versus
        h_versus = teams.index(a_team)
        a_versus = teams.index(h_team)
        
        # Defense (points let in)
        h_team_defense = a_score
        a_team_defense = h_score  
        
        # Attack Points scored
        h_team_attack = h_score 
        a_team_attack = a_score   
        
        # Margin
        h_team_margin =  h_score-a_score   
        a_team_margin =  a_score-h_score        
        
        # Keep track of which teams played to work out which teams had a bye 
        round_teams.append(h_team)
        round_teams.append(a_team)
        
        
        # Find the index of the team in the overaching array 
        a_team_idx = teams.index(a_team)
        h_team_idx = teams.index(h_team)
        
        # Feature map index
        a_team_idx_fm = a_team_idx*len(variables)
        h_team_idx_fm = h_team_idx*len(variables)
        
        # input(round_store)
        # input(a_team_idx_fm)
        # populate the data
        round_store[a_team_idx_fm] = a_team_win
        round_store[a_team_idx_fm+1] = a_team_defense
        round_store[a_team_idx_fm+2] = a_team_attack
        round_store[a_team_idx_fm+3] = a_team_margin
        round_store[a_team_idx_fm+4] = a_home
        round_store[a_team_idx_fm+5] = a_versus
        round_store[h_team_idx_fm] = h_team_win
        round_store[h_team_idx_fm+1] = h_team_defense
        round_store[h_team_idx_fm+2] = h_team_attack
        round_store[h_team_idx_fm+3] = h_team_margin
        round_store[h_team_idx_fm+4] = h_home
        round_store[h_team_idx_fm+5] = h_versus
        
    # input(round_teams)
    bye_teams = list(set(teams) - set(round_teams))
    for bye_team in bye_teams:
        b_team_idx = teams.index(bye_team)
        b_team_idx_fm = b_team_idx*len(variables)
        round_store[b_team_idx_fm] = -1
        round_store[b_team_idx_fm+1] = -1
        round_store[b_team_idx_fm+2] = -1
        round_store[b_team_idx_fm+3] = 0
        round_store[b_team_idx_fm+4] = -1
        round_store[b_team_idx_fm+5] = -1
        
    all_store.append(round_store)
    # Create an array with the same size as the DataFrame

    # Add the new row to the DataFrame using loc
    df.loc[len(df)] = round_store
df

,Broncos Win,Broncos Defense,Broncos Attack,Broncos Margin,Broncos Home,Broncos Versus,Roosters Win,Roosters Defense,Roosters Attack,Roosters Margin,...,Cowboys Attack,Cowboys Margin,Cowboys Home,Cowboys Versus,Warriors Win,Warriors Defense,Warriors Attack,Warriors Margin,Warriors Home,Warriors Versus
0,1,12,13,1,0,10,0,28,18,-10,...,19,1,1,6,1,12,20,8,1,7
1,1,16,28,12,1,15,1,12,20,8,...,16,-12,0,0,0,20,12,-8,0,1
2,1,18,40,22,1,8,1,18,20,2,...,12,-14,1,16,1,12,26,14,0,15
3,1,12,18,6,0,13,-1,-1,-1,0,...,24,12,1,14,1,14,16,2,1,12
4,1,12,46,34,1,2,1,20,28,8,...,14,-1,0,12,1,30,32,2,0,11
5,0,20,14,-6,1,6,0,28,8,-20,...,22,-10,1,13,0,34,24,-10,0,7
6,1,26,43,17,0,14,0,22,12,-10,...,14,-8,0,16,1,14,22,8,1,15
7,1,16,26,10,0,5,1,26,27,1,...,18,2,1,7,0,30,22,-8,0,4
8,0,32,6,-26,1,3,1,0,14,14,...,6,-38,0,11,0,14,0,-14,1,1
9,1,6,32,26,0,9,0,20,6,-14,...,20,14,0,1,0,18,6,-12,1,10


In [28]:
GAME_HISTORY = 5

def get_game_history(round, team):
    win = df[team+ " Win"][round-GAME_HISTORY:round].mean()
    defense = df[team+ " Defense"][round-GAME_HISTORY:round].mean()
    attack = df[team+ " Attack"][round-GAME_HISTORY:round].mean()
    margin = df[team+ " Margin"][round-GAME_HISTORY:round].mean()
    return win, defense, attack, margin
    

In [107]:
input = []
output = [] 
# Input: Team, Other Team Team Stats, Other Team Stats
# Output: Team, win/lose, other team, win/lose


for team in teams:
    # Skip the game history as we dont want to break the calibration
    versed_teams = df[team+ " Versus"][GAME_HISTORY:-1]
    wins = df[team+ " Win"][GAME_HISTORY:-1]
    c_team_idx = teams.index(team)
    for round, (versed_team, win) in enumerate(zip(versed_teams, wins)):
        c_win, c_defense, c_attack, c_margin = get_game_history(round+GAME_HISTORY+1, team)
        v_win, v_defense, v_attack, v_margin = get_game_history(round+GAME_HISTORY+1, teams[versed_team])
        
        # dosent take into account for ties, look into this harder later 
        if win == 1:
            v_win_ = 0
        else:
            v_win_ = 1
        
        # Current team, vsed team, who wins, current_team_stats, vsed_team_stats
        input.append([c_team_idx, versed_team, c_win, c_defense, c_attack, c_margin, v_win, v_defense, v_attack, v_margin])
        output.append([c_team_idx, versed_team, win, v_win_])


In [ ]:
!pip install scikit-learn

In [108]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(input, output, test_size=0.2, random_state=42)
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)



LinearRegression()

In [ ]:
# input_end = []
# output_end = [] 
# # Input: Team, Other Team Team Stats, Other Team Stats
# # Output: Team, win/lose, other team, win/lose


# for team in teams:
#     # Skip the game history as we dont want to break the calibration
#     versed_team = df[team+ " Versus"].iloc[-1]
    
#     win = df[team+ " Win"].iloc[-1]
#     c_team_idx = teams.index(team)
#     c_win, c_defense, c_attack, c_margin = get_game_history(22, team)
#     v_win, v_defense, v_attack, v_margin = get_game_history(22, teams[versed_team])
    
#     # dosent take into account for ties, look into this harder later 
#     if win == 1:
#         v_win = 0
#     else:
#         v_win = 1
    
#     # Current team, vsed team, who wins, current_team_stats, vsed_team_stats
#     input_end.append([c_team_idx, versed_team, c_win, c_defense, c_attack, c_margin, v_win, v_defense, v_attack, v_margin])
#     output_end.append([c_team_idx, win, versed_team, v_win])
        
# input_end

In [109]:
# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.11664099010458431


In [112]:
# teams = ["Broncos", "Roosters", "Wests Tigers", "Rabbitohs", "Storm", "Eels", "Raiders", "Knights", "Dragons", "Sea Eagles", "Panthers", "Sharks", "Bulldogs", "Dolphins", "Titans", "Cowboys", "Warriors"]

wkd_matches = [["Dragons", "Wests Tigers"], ["Warriors", "Raiders"], ["Rabbitohs", "Broncos"], ["Titans", "Roosters"], ["Knights", "Storm"], ["Cowboys", "Eels"], ["Panthers", "Bulldogs"], ["Sharks", "Sea Eagles"]]

for wkd_match in wkd_matches:

    team_1 = int(teams.index(wkd_match[0]))
    team_2 = int(teams.index(wkd_match[1]))
    # input(team_1)

    c_win, c_defense, c_attack, c_margin = get_game_history(22, teams[int(team_1)])
    v_win, v_defense, v_attack, v_margin =  get_game_history(22, teams[int(team_2)])
    predictions = model.predict([[team_1, team_2, c_win, c_defense, c_attack, c_margin, v_win, v_defense, v_attack, v_margin]])
    predictions = predictions[0]
    if predictions[2] > predictions[3]:
        print(f"{teams[team_1]} wins")
    else:
        print(f"{teams[team_2]} wins")


Dragons wins
Warriors wins
Broncos wins
Roosters wins
Knights wins
Cowboys wins
Panthers wins
Sharks wins
